In [ ]:
import os
import pandas as pd
import numpy as np
from datetime import timedelta


YEAR = 2024          # we are considering primary dataset for 2024
NUM_PRODUCTS = 50
OUT_DIR = "data"
OUT_FILE = os.path.join(OUT_DIR, "synthetic_weekly_sales.csv")

os.makedirs(OUT_DIR, exist_ok=True)


#  Find all Sundays in the selected year 
jan1 = pd.to_datetime(f"{YEAR}-01-01")
dec31 = pd.to_datetime(f"{YEAR}-12-31")

# finding First Sunday ON or AFTER Jan 1
days_to_first_sunday = (6 - jan1.weekday()) % 7   # weekday(): Monday=0 ... Sunday=6
first_sunday = jan1 + pd.Timedelta(days=days_to_first_sunday)

# Last Sunday ON or BEFORE Dec 31
days_since_last_sunday = (dec31.weekday() - 6) % 7
last_sunday = dec31 - pd.Timedelta(days=days_since_last_sunday)

# Generate weekly Sundays for the year
week_dates = pd.date_range(start=first_sunday, end=last_sunday, freq="W-SUN")
print(f"Total weeks in {YEAR}: {len(week_dates)}")
print(f"From {week_dates.min().date()} to {week_dates.max().date()}")


#  creating the product definitions which is product_id and product_names
product_ids = [f"P{i:03d}" for i in range(1, NUM_PRODUCTS + 1)]
product_names = [f"Product_{i}" for i in range(1, NUM_PRODUCTS + 1)]

np.random.seed(42)
base_prices = np.random.randint(10, 101, size=NUM_PRODUCTS)
base_units  = np.random.randint(50, 501, size=NUM_PRODUCTS)


#  Generating  synthetic data for 2024
data = []

for week in week_dates:
    week_num = week.isocalendar()[1]
    for pid, pname, price, units_base in zip(product_ids, product_names, base_prices, base_units):

        # adding seasonality factor, considering 0.5
        seasonal_factor = 1 + 0.3 * np.sin(2 * np.pi * week_num / 52)

        # discount 0–30% with weighted randomness
        discount = np.random.choice([0,5,10,15,20,25,30], p=[0.4,0.1,0.1,0.1,0.1,0.1,0.1])

        # units sold affected by seasonality and discount
        units_sold = int(units_base * seasonal_factor * (1 + discount/100 * 0.5))

        # total revenue generated
        revenue = units_sold * price * (1 - discount/100)

        data.append({
            "Week_Start_Date": week.normalize().date(),
            "Product_ID": pid,
            "Product_Name": pname,
            "Units_Sold": units_sold,
            "Price": int(price),
            "Discount_Percentage": int(discount),
            "Revenue": round(float(revenue), 2),
            "Region": "USA"
        })

df_sales = pd.DataFrame(data)


# saving output to csv file 
df_sales.to_csv(OUT_FILE, index=False)

print("\n✔ Synthetic weekly sales dataset created!")
print("Saved to:", OUT_FILE)
df_sales.head()


Total weeks in 2024: 52
From 2024-01-07 to 2024-12-29

✔ Synthetic weekly sales dataset created!
Saved to: data\synthetic_weekly_sales.csv


,Week_Start_Date,Product_ID,Product_Name,Units_Sold,Price,Discount_Percentage,Revenue,Region
0,2024-01-07,P001,Product_1,434,61,30,18531.8,USA
1,2024-01-07,P002,Product_2,70,24,15,1428.0,USA
2,2024-01-07,P003,Product_3,301,81,0,24381.0,USA
3,2024-01-07,P004,Product_4,325,70,0,22750.0,USA
4,2024-01-07,P005,Product_5,409,30,0,12270.0,USA


In [ ]:
#importing the API key from .env file by importing dotenv function
from dotenv import load_dotenv
import os

load_dotenv("api.env")

FRED_API_KEY = os.getenv("FRED_API_KEY")
print(FRED_API_KEY)

627b7861683c0e8883cf65f517644f83


In [ ]:
import requests
import pandas as pd
import numpy as np

def fetch_fred_series(series_id, start_date=None, end_date=None, api_key=FRED_API_KEY):
    url = "https://api.stlouisfed.org/fred/series/observations"
    params = {
        "series_id": series_id,
        "api_key": api_key,
        "file_type": "json",
    }
     
    if start_date:
        params["observation_start"] = start_date
    if end_date:
        params["observation_end"] = end_date

    resp = requests.get(url, params=params)
    resp.raise_for_status()

    data = resp.json().get("observations", [])
    df = pd.DataFrame(data)
    df["date"] = pd.to_datetime(df["date"])
    df[series_id] = pd.to_numeric(df["value"].replace('.', np.nan))
# Return cleaned DataFrame with only two useful columns
    return df[["date", series_id]]


In [ ]:
# Determine the date range of the sales dataset.
# We use this range to fetch matching economic indicators.
start = df_sales["Week_Start_Date"].min().strftime("%Y-%m-%d")
end   = df_sales["Week_Start_Date"].max().strftime("%Y-%m-%d")
# Fetch weekly U.S. gasoline price data from FRED for the same date window.
df_gas = fetch_fred_series("GASREGW", start, end)
# Fetch Consumer Price Index (CPI) data for the same period.
df_cpi = fetch_fred_series("CPIAUCSL", start, end)
# Display the first few rows of each dataset to verify the API fetch worked correctly.
df_gas.head(), df_cpi.head()


(        date  GASREGW
 0 2024-01-08    3.073
 1 2024-01-15    3.058
 2 2024-01-22    3.062
 3 2024-01-29    3.095
 4 2024-02-05    3.136,
         date  CPIAUCSL
 0 2024-01-01   309.794
 1 2024-02-01   311.022
 2 2024-03-01   312.107
 3 2024-04-01   313.016
 4 2024-05-01   313.140)

In [ ]:
# we need to df_sales is loaded and Week_Start_Date is datetime
df_sales['Week_Start_Date'] = pd.to_datetime(df_sales['Week_Start_Date']).dt.normalize()

start = df_sales["Week_Start_Date"].min().strftime("%Y-%m-%d")
end   = df_sales["Week_Start_Date"].max().strftime("%Y-%m-%d")

# Weekly Sundays index matching our sales
weekly_index = pd.date_range(start=start, end=end, freq="W-SUN")
print("Weeks:", weekly_index[0], "->", weekly_index[-1], "count:", len(weekly_index))


Weeks: 2024-01-07 00:00:00 -> 2024-12-29 00:00:00 count: 52


In [ ]:
# Sort the FRED gas price data by date and set the date column as the index
df_gas = df_gas.sort_values('date').set_index('date')

# Resample to weekly Sundays (aggregate by mean if multiple), then reindex to exact weekly_index
df_gas_weekly = df_gas.resample('W-SUN').mean().reindex(weekly_index)

# Interpolate small gaps in gas price if any
df_gas_weekly['GASREGW'] = df_gas_weekly['GASREGW'].interpolate(method='time')

# Prepare final frame
df_gas_weekly = df_gas_weekly.rename_axis('Week_Start_Date').reset_index().rename(columns={'GASREGW':'Avg_Gas_Price_US'})
# Preview the final weekly gas-price dataset
df_gas_weekly.head()


,Week_Start_Date,Avg_Gas_Price_US
0,2024-01-07,NaN
1,2024-01-14,3.073
2,2024-01-21,3.058
3,2024-01-28,3.062
4,2024-02-04,3.095


In [7]:
# df_cpi has columns ['date','CPIAUCSL']
df_cpi = df_cpi.sort_values('date').set_index('date')

# Create a daily index from earliest CPI date to latest, forward-fill monthly CPI to each day
daily_cpi = df_cpi.reindex(pd.date_range(df_cpi.index.min(), df_cpi.index.max(), freq='D')).fillna(method='ffill')

# Resample daily forward-filled CPI to weekly Sundays (take last day of week)
df_cpi_weekly = daily_cpi.resample('W-SUN').last().reindex(weekly_index).ffill()

# Prepare final frame
df_cpi_weekly = df_cpi_weekly.rename_axis('Week_Start_Date').reset_index().rename(columns={'CPIAUCSL':'CPI'})
df_cpi_weekly.head()


C:\Users\mkman\AppData\Local\Temp\ipykernel_27408\2108556893.py:5: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  daily_cpi = df_cpi.reindex(pd.date_range(df_cpi.index.min(), df_cpi.index.max(), freq='D')).fillna(method='ffill')


,Week_Start_Date,CPI
0,2024-01-07,309.794
1,2024-01-14,309.794
2,2024-01-21,309.794
3,2024-01-28,309.794
4,2024-02-04,311.022


In [8]:
# Normalize date columns
df_gas_weekly['Week_Start_Date'] = pd.to_datetime(df_gas_weekly['Week_Start_Date']).dt.normalize()
df_cpi_weekly['Week_Start_Date'] = pd.to_datetime(df_cpi_weekly['Week_Start_Date']).dt.normalize()

# Merge (left join sales → econ)
df_merged = df_sales.merge(df_gas_weekly[['Week_Start_Date','Avg_Gas_Price_US']], on='Week_Start_Date', how='left')
df_merged = df_merged.merge(df_cpi_weekly[['Week_Start_Date','CPI']], on='Week_Start_Date', how='left')

print("Merged shape:", df_merged.shape)
df_merged.head()


Merged shape: (2600, 10)


,Week_Start_Date,Product_ID,Product_Name,Units_Sold,Price,Discount_Percentage,Revenue,Region,Avg_Gas_Price_US,CPI
0,2024-01-07,P001,Product_1,434,61,30,18531.8,USA,NaN,309.794
1,2024-01-07,P002,Product_2,70,24,15,1428.0,USA,NaN,309.794
2,2024-01-07,P003,Product_3,301,81,0,24381.0,USA,NaN,309.794
3,2024-01-07,P004,Product_4,325,70,0,22750.0,USA,NaN,309.794
4,2024-01-07,P005,Product_5,409,30,0,12270.0,USA,NaN,309.794


In [9]:
!python automate.py

Traceback (most recent call last):
  File "d:\folder_project\automate.py", line 27, in <module>
    from scipy import stats
ModuleNotFoundError: No module named 'scipy'


In [10]:
pip install scipy


   ---------------------------------------- 0.0/38.5 MB ? eta -:--:--
   --- ------------------------------------ 3.4/38.5 MB 18.6 MB/s eta 0:00:02
   ---- ----------------------------------- 4.2/38.5 MB 10.7 MB/s eta 0:00:04
   ---- ----------------------------------- 4.5/38.5 MB 8.4 MB/s eta 0:00:05
   ----- ---------------------------------- 5.0/38.5 MB 6.0 MB/s eta 0:00:06
   ----- ---------------------------------- 5.5/38.5 MB 5.5 MB/s eta 0:00:07
   ------ --------------------------------- 6.3/38.5 MB 4.8 MB/s eta 0:00:07
   ------- -------------------------------- 6.8/38.5 MB 4.6 MB/s eta 0:00:07
   -------- ------------------------------- 7.9/38.5 MB 4.6 MB/s eta 0:00:07
   --------- ------------------------------ 9.2/38.5 MB 4.8 MB/s eta 0:00:07
   ---------- ----------------------------- 10.0/38.5 MB 4.8 MB/s eta 0:00:06
   ---------- ----------------------------- 10.5/38.5 MB 4.6 MB/s eta 0:00:07
   ----------- ---------------------------- 11.3/38.5 MB 4.5 MB/s eta 0:00:07
 


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
!python automate.py

ALERTS FOUND:
 Missing critical values: {'Week_Start_Date': 0, 'Product_ID': 0, 'Units_Sold': 0, 'Price': 0, 'Revenue': 0, 'Avg_Gas_Price_US': 50, 'CPI': 0}

Economic indicators missing counts: {'Avg_Gas_Price_US': 50, 'CPI': 0}
Alert email sent.


d:\folder_project\automate.py:287: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  daily = df_cpi_idx.reindex(pd.date_range(df_cpi_idx.index.min(), df_cpi_idx.index.max(), freq="D")).fillna(method='ffill')
